Test du projet de class trajets indirect

In [1]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\Martin\git\Outils\Outils\Martin_Perso')
import traitement_od as t
import fiona
import pandas as pd
import numpy as np
import altair as alt
import datetime as dt

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 85: invalid continuation byte

Exception ignored in: 'fiona._env.log_error'
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 85: invalid continuation byte


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 85: invalid continuation byte

Exception ignored in: 'fiona._env.log_error'
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 85: invalid continuation byte


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 86: invalid continuation byte

...

Exception ignored in: 'fiona._env.log_error'
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 85: invalid continuation byte


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 88: invalid continuation byte

Exception ignored in: 'fiona._env.log_error'
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 88: invalid continuation byte


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 88: invalid continuation byte

Exception ignored in: 'fiona._env.log_error'
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 88: invalid continuation byte


In [10]:
df_journee=t.ouvrir_fichier_lapi('2019-01-29 00:00:00','2019-01-30 23:59:00')

In [65]:
toto=t.trajet_indirect(df_journee,'2019-01-29 00:00:00', 60, 16, [19,4,5])

In [66]:
toto.df_transit.empty

True

In [24]:
liste_trajet=(pd.DataFrame({'o_d':'A63-A10','trajets':[[{'cameras':[19,4,5],'type_trajet':'indirect'},
                                                       {'cameras':[19,1,5],'type_trajet':'indirect'},
                                                       {'cameras':[19,5],'type_trajet':'direct'},
                                                      ]]}))

In [69]:
dates= pd.date_range('2019-01-29', periods=24, freq='H')
for date in dates : 
    date=date.strftime("%Y-%m-%d %H:%M:%S")
    for index, value in liste_trajet.iterrows() : 
        o_d, carac_trajet=value[0],value[1]
        for dico_carac in carac_trajet :
            cameras=dico_carac['cameras']
            type_t=dico_carac['type_trajet']
            if type_t=='indirect' : 
                df_trajet=t.trajet_indirect(df_journee,date, 60, 16, cameras).df_transit#en deduire le total
            else :
                df_trajet=t.df_tps_parcours(df_journee,date, 60, 16, cameras[0],cameras[1]).df_tps_parcours_pl_final#por permettre le suivi et retrouver les camera intermediaires
            if df_trajet.empty : 
                continue
            df_trajet['cameras']=str(cameras)#pour suivi possible
            if 'dico_od' in locals() :
                dico_od=pd.concat([dico_od,df_trajet], sort=False)
            else : 
                dico_od=df_trajet
            print(date, cameras, len(df_trajet))

2019-01-29 07:00:00 [19, 4, 5] 9
2019-01-29 07:00:00 [19, 5] 1
2019-01-29 08:00:00 [19, 4, 5] 86
2019-01-29 08:00:00 [19, 1, 5] 2
2019-01-29 08:00:00 [19, 5] 15
2019-01-29 09:00:00 [19, 4, 5] 78
2019-01-29 09:00:00 [19, 1, 5] 2
2019-01-29 09:00:00 [19, 5] 14
2019-01-29 10:00:00 [19, 4, 5] 43
2019-01-29 10:00:00 [19, 1, 5] 2
2019-01-29 10:00:00 [19, 5] 25
2019-01-29 11:00:00 [19, 4, 5] 34
2019-01-29 11:00:00 [19, 1, 5] 1
2019-01-29 11:00:00 [19, 5] 28
2019-01-29 12:00:00 [19, 5] 82
2019-01-29 13:00:00 [19, 4, 5] 5
2019-01-29 13:00:00 [19, 5] 64
2019-01-29 14:00:00 [19, 4, 5] 46
2019-01-29 14:00:00 [19, 1, 5] 3
2019-01-29 14:00:00 [19, 5] 36
2019-01-29 15:00:00 [19, 4, 5] 31
2019-01-29 15:00:00 [19, 1, 5] 2
2019-01-29 15:00:00 [19, 5] 14
2019-01-29 16:00:00 [19, 5] 1
2019-01-29 17:00:00 [19, 4, 5] 1
